In [1]:
from sqlalchemy import create_engine
import pandas as pd
from urllib.parse import quote_plus
import numpy as np
# Conexión a la base de datos MySQL
usuario = "root"
contraseña = "3112708652El"
host = "localhost"
puerto = 3306
basededatos = "demo_wft"

contraseña_escapada = quote_plus(contraseña)

engine = create_engine(f"mysql+pymysql://{usuario}:{contraseña_escapada}@{host}:{puerto}/{basededatos}")

df = pd.read_sql("SELECT * FROM accounting_account_balances", engine)

df['currency_id'] = df['currency_id'].replace('', np.nan)
df['currency_id'] = df['currency_id'].fillna('COP')

print(df.head())

   id  code  accounting_id    name  initial_balance  final_balance  \
0   1   1.0              1  Activo              0.0  -4.500000e+04   
1   2   1.0              1  Activo              0.0   3.550000e+06   
2   3   1.0              1  Activo              0.0   0.000000e+00   
3   4   1.0              1  Activo              0.0   3.730769e+03   
4   5   1.0              1  Activo              0.0   2.781800e+04   

   debit_movement  credit_movement third_party_type_id  third_party_id  \
0       3525000.0     3.570000e+06             Contact               1   
1       3570000.0     2.000000e+04             Contact               1   
2         59500.0     5.950000e+04             Contact              13   
3         59500.0     5.576923e+04             Contact              13   
4        149940.0     1.221220e+05             Contact              34   

  currency_id  year  month deleted_at          created_at          updated_at  
0         COP  2024      2       None 2024-02-29 19:00

In [2]:
from sqlalchemy import inspect

# Obtener los nombres de todas las tablas de la base de datos conectadas
inspector = inspect(engine)
tablas = inspector.get_table_names()
df_tablas = pd.DataFrame(tablas, columns=['Nombre de la tabla'])
df_tablas

,Nombre de la tabla
0,accounting_account_balances
1,accounting_accounts
2,accounting_configurations
3,accounting_movements
4,accounting_voucher_items
...,...
111,user_headquarters
112,user_roles
113,values_x_item
114,warehouse_transfer_logs


In [23]:
# ...existing code...

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

# ================================
# 1. Obtener y Procesar Ingresos
# ================================

# Fechas desde enero 2024 hasta mayo 2025
fechas = pd.date_range(start='2024-01-01', end='2025-05-01', freq='MS')

# Simular ingresos realistas
ingresos = [
    8200, 8500, 8800, 8900, 9100, 9400, 9700, 10000, 9800, 9500, 9700, 9900,  # 2024
    6000, 5500, 5000, 4800, 4700  # 2025 (baja progresiva)
]

df_ingresos = pd.DataFrame({'mes': fechas, 'ingresos': ingresos})

# ================================
# 2. Simular COGS (Costo de Bienes Vendidos) realista
# ================================

# El COGS suele estar entre 60% y 75% de los ingresos, pero puede variar mes a mes
np.random.seed(123)
porcentajes_cogs = np.random.uniform(0.60, 0.75, len(fechas)).round(2)
cogs = (df_ingresos['ingresos'] * porcentajes_cogs).astype(int)

df_cogs = pd.DataFrame({'mes': fechas, 'cogs': cogs})

# ================================
# 3. Simular Gastos (altos pero no tanto, algunos meses > ingresos)
# ================================

np.random.seed(99)
# Gastos entre 45% y 85% de los ingresos, para que algunos meses sean negativos pero no todos
porcentajes_gastos = np.random.uniform(0.45, 0.85, len(fechas)).round(2)
# Forzar algunos meses a ser mayores que ingresos
porcentajes_gastos[2] = 1.05   # Marzo 2024
porcentajes_gastos[7] = 1.10   # Agosto 2024
porcentajes_gastos[13] = 1.02  # Febrero 2025

gastos = (df_ingresos['ingresos'] * porcentajes_gastos).astype(int)
df_gastos = pd.DataFrame({'mes': fechas, 'gastos': gastos})

# ================================
# 4. Unir Ingresos, COGS y Gastos
# ================================

df_ventas = df_ingresos.merge(df_cogs, on='mes').merge(df_gastos, on='mes')

# Calcular utilidad bruta, margen bruto, utilidad neta y margen neto
df_ventas['utilidad_bruta'] = df_ventas['ingresos'] - df_ventas['cogs']
df_ventas['margen_bruto'] = (df_ventas['utilidad_bruta'] / df_ventas['ingresos']).round(2)
df_ventas['utilidad_neta'] = df_ventas['utilidad_bruta'] - df_ventas['gastos']
df_ventas['margen_neto'] = (df_ventas['utilidad_neta'] / df_ventas['ingresos']).round(2)

# ================================
# 5. Gráfico Ingresos vs Gastos
# ================================

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_ventas['mes'],
    y=df_ventas['ingresos'],
    name='Ingresos',
    marker_color='#28a745'
))

fig.add_trace(go.Bar(
    x=df_ventas['mes'],
    y=df_ventas['gastos'],
    name='Gastos',
    marker_color='#dc3545'
))

fig.update_layout(
    title='Ingresos vs Gastos Mensuales (2024-01 a 2025-05)',
    xaxis_title='Mes',
    yaxis_title='Monto ($)',
    barmode='group',
    xaxis=dict(
        tickvals=df_ventas['mes'],
        ticktext=[d.strftime('%b %Y') for d in df_ventas['mes']],
        tickangle=45
    ),
    template='plotly_white'
)

fig.show()

# ================================
# 6. Tendencia de Márgenes Bruto y Neto
# ================================

fig_margenes = go.Figure()
fig_margenes.add_trace(go.Scatter(
    x=df_ventas['mes'],
    y=df_ventas['margen_bruto'],
    mode='lines+markers',
    name='Margen Bruto',
    line=dict(color='#007bff', width=3)
))
fig_margenes.add_trace(go.Scatter(
    x=df_ventas['mes'],
    y=df_ventas['margen_neto'],
    mode='lines+markers',
    name='Margen Neto',
    line=dict(color='#ff9900', width=3)
))
fig_margenes.update_layout(
    title='Tendencia de Márgenes Bruto y Neto (2024-01 a 2025-05)',
    xaxis_title='Mes',
    yaxis_title='Margen',
    yaxis_tickformat='.0%',
    xaxis=dict(
        tickvals=df_ventas['mes'],
        ticktext=[d.strftime('%b %Y') for d in df_ventas['mes']],
        tickangle=45
    ),
    template='plotly_white'
)
fig_margenes.show()

# ================================
# 7. Márgenes Promedio y Gauge con flecha indicadora
# ================================

margen_bruto_promedio = df_ventas['margen_bruto'].mean()
margen_neto_promedio = df_ventas['margen_neto'].mean()
print(f'Margen Bruto Promedio: {margen_bruto_promedio:.2%}')
print(f'Margen Neto Promedio: {margen_neto_promedio:.2%}')

fig_gauge = go.Figure(go.Indicator(
    mode="gauge+number+delta",
    value=margen_bruto_promedio,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Margen Bruto Promedio"},
    gauge={
        'axis': {'range': [0, 1]},
        'bar': {'color': "#007bff"},
        'steps': [
            {'range': [0, 0.5], 'color': "#ffcccc"},
            {'range': [0.5, 0.7], 'color': "#ffe066"},
            {'range': [0.7, 1], 'color': "#ccffcc"}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': margen_bruto_promedio
        }
    },
    number={'valueformat': '.2%'}
))
fig_gauge.show()
# ...existing code...

Margen Bruto Promedio: 32.71%
Margen Neto Promedio: -42.00%
